# Multiparty Linkage with Clkhash


## Scenario

There are three parties named Alice, Bob, and Charlie, each holding a dataset of about 3200 records. They know that they have some entities in common, but with incomplete overlap. The common features describing those entities are given name, surname, date of birth, and phone number.

They all have some additional information about those entities in their respective datasets, Alice has a person's gender, Bob has their city, and Charlie has their income. They wish to create a table for analysis: each row has a gender, city, and income, but they don't want to share any additional information. They can use Anonlink to do this in a privacy-preserving way (without revealing given names, surnames, dates of birth, and phone numbers).

In [1]:
import csv
import json
import os

import pandas as pd

## Alice, Bob, and Charlie: agree on secret keys and a linkage schema

They keep the keys to themselves, but the schema may be revealed to the analyst.

In [2]:
SECRET = 'my_secret'

SERVER = os.getenv("SERVER", "https://anonlink.easd.data61.xyz")

In [3]:
with open('data/schema_ABC.json') as f:
    print(f.read())

{
  "version": 3,
  "clkConfig": {
    "l": 1024,
    "kdf": {
      "type": "HKDF",
      "hash": "SHA256",
      "salt": "SCbL2zHNnmsckfzchsNkZY9XoHk96P/G5nUBrM7ybymlEFsMV6PAeDZCNp3rfNUPCtLDMOGQHG4pCQpfhiHCyA==",
      "info": "c2NoZW1hX2V4YW1wbGU=",
      "keySize": 64
    }
  },
  "features": [
    {
      "identifier": "id",
      "ignored": true
    },
    {
      "identifier": "givenname",
      "format": {
        "type": "string",
        "encoding": "utf-8"
      },
      "hashing": {
        "strategy": {
          "bitsPerToken": 15
        },
        "comparison": {
          "type": "ngram",
          "n": 2,
          "positional": false
        }
      }
    },
    {
      "identifier": "surname",
      "format": {
        "type": "string",
        "encoding": "utf-8"
      },
      "hashing": {
        "strategy": {
          "bitsPerToken": 15
        },
        "comparison": {
          "type": "ngram",
          "n": 2,
          "positional": false
        }
      

## Sneak peek at input data

### Alice

In [4]:
pd.read_csv('data/dataset-alice.csv').head()

,id,givenname,surname,dob,phone number,gender
0,0,tara,hilton,27-08-1941,08 2210 0298,male
1,3,saJi,vernre,22-12-2972,02 1090 1906,mals
2,7,sliver,paciorek,NaN,NaN,mals
3,9,ruby,george,09-05-1939,07 4698 6255,male
4,10,eyrinm,campbell,29-1q-1983,08 299y 1535,male


### Bob

In [5]:
pd.read_csv('data/dataset-bob.csv').head()

,id,givenname,surname,dob,phone number,city
0,3,zali,verner,22-12-1972,02 1090 1906,perth
1,4,samuel,tremellen,21-12-1923,03 3605 9336,melbourne
2,5,amy,lodge,16-01-1958,07 8286 9372,canberra
3,7,oIji,pacioerk,10-02-1959,04 4220 5949,sydney
4,10,erin,kampgell,29-12-1983,08 2996 1445,perth


## Charlie

In [6]:
pd.read_csv('data/dataset-charlie.csv').head()

,id,givenname,surname,dob,phone number,income
0,1,joshua,arkwright,16-02-1903,04 8511 9580,70189.446
1,3,zal:,verner,22-12-1972,02 1090 1906,50194.118
2,7,oliyer,paciorwk,10-02-1959,04 4210 5949,31750.993
3,8,nacoya,ranson,17-08-1925,07 6033 4580,102446.131
4,10,erih,campbell,29-12-1i83,08 299t 1435,331476.599


## Analyst: create the project

The analyst keeps the result token to themselves. The three update tokens go to Alice, Bob and Charlie. The project ID is known by everyone.

In [7]:
!anonlink create-project \
    --server $SERVER \
    --type groups \
    --schema data/schema_ABC.json \
    --parties 3 \
    --output credentials.json

with open('credentials.json') as f:
    credentials = json.load(f)
    project_id = credentials['project_id']
    result_token = credentials['result_token']
    update_token_alice = credentials['update_tokens'][0]
    update_token_bob = credentials['update_tokens'][1]
    update_token_charlie = credentials['update_tokens'][2]

Project created


## Alice: hash the data and upload it to the server
The data is hashed according to the schema and the keys. Alice's update token is needed to upload the hashed data. No PII is uploaded to the service—only the hashes.

In [8]:
!anonlink hash \
    data/dataset-alice.csv \
    $SECRET \
    data/schema_ABC.json \
    dataset-alice-hashed.json \
    --check-header false

CLK data written to dataset-alice-hashed.json


In [9]:
!anonlink upload \
    --server $SERVER \
    --apikey $update_token_alice \
    --project $project_id \
    dataset-alice-hashed.json

{"message": "Updated", "receipt_token": "8fb72ac2461038c3d83143890c7a741317775ddc8781a8bf"}

## Bob: hash the data and upload it to the server

In [10]:
!anonlink hash \
    data/dataset-bob.csv \
    $SECRET \
    data/schema_ABC.json \
    dataset-bob-hashed.json \
    --check-header false

CLK data written to dataset-bob-hashed.json


In [11]:
!anonlink upload \
    --server $SERVER \
    --apikey $update_token_bob \
    --project $project_id \
    dataset-bob-hashed.json

{"message": "Updated", "receipt_token": "12974f0a758e096001dee76c1b94e065ab2e17cd4992f490"}

## Charlie: hash the data and upload it to the server

In [12]:
!anonlink hash \
    data/dataset-charlie.csv \
    $SECRET \
    data/schema_ABC.json \
    dataset-charlie-hashed.json \
    --check-header false

CLK data written to dataset-charlie-hashed.json


In [13]:
!anonlink upload \
    --server $SERVER \
    --apikey $update_token_charlie \
    --project $project_id \
    dataset-charlie-hashed.json

{"message": "Updated", "receipt_token": "6f169da70b230132767999759f77e1bd8ad40e7ea363a3d6"}

## Analyst: start the linkage run

This will start the linkage computation. We will wait a little bit and then retrieve the results.

In [14]:
!anonlink create \
    --server $SERVER \
    --project $project_id \
    --apikey $result_token \
    --threshold 0.7 \
    --output=run-credentials.json

with open('run-credentials.json') as f:
    run_credentials = json.load(f)
    run_id = run_credentials['run_id']

## Analyst: retrieve the results

In [15]:
!anonlink results \
    --server $SERVER \
    --project $project_id \
    --apikey $result_token \
    --run $run_id \
    --watch \
    --output linkage-output.json

State: running
Stage (2/3): compute similarity scores
Progress: 33.19%
State: running
Stage (2/3): compute similarity scores
Progress: 33.19%
State: running
Stage (3/3): compute output
State: completed
Stage (3/3): compute output
Received result


In [16]:
with open('linkage-output.json') as f:
    linkage_output = json.load(f)
    linkage_groups = linkage_output['groups']
linkage_groups[-15:]

[[[1, 1014], [2, 1066], [0, 1046]],
 [[1, 1961], [2, 1982], [0, 1989]],
 [[0, 2479], [2, 2508], [1, 2492]],
 [[0, 3167], [2, 3194]],
 [[0, 154], [1, 151], [2, 158]],
 [[1, 1941], [2, 1961]],
 [[0, 3126], [2, 3151], [1, 3142]],
 [[0, 30], [1, 34], [2, 44]],
 [[1, 2918], [2, 2909]],
 [[0, 804], [1, 946]],
 [[0, 1033], [2, 1049], [1, 1002]],
 [[0, 2071], [2, 2070], [1, 2052]],
 [[0, 3183], [1, 3197]],
 [[0, 2689], [1, 2695], [2, 2702]],
 [[1, 3067], [2, 3078]]]

The result is a list of groups of records. Every record in such a group belongs to the same entity and consists of two values, the party index and the row index:
```
[
  [[party_id, row_index], ... ],
  ...
]
```

## Everyone: make table of interesting information

We use the linkage result to make a table of genders, cities, and incomes without revealing any other PII.

In [17]:
with open('data/dataset-alice.csv') as f:
    r = csv.reader(f)
    next(r)  # Skip header
    genders = tuple(row[-1] for row in r)
    
with open('data/dataset-bob.csv') as f:
    r = csv.reader(f)
    next(r)  # Skip header
    cities = tuple(row[-1] for row in r)
    
with open('data/dataset-charlie.csv') as f:
    r = csv.reader(f)
    next(r)  # Skip header
    incomes = tuple(row[-1] for row in r)

In [18]:
table = []
for group in linkage_groups:
    row = [''] * 3
    for i, j in group:
        row[i] = [genders, cities, incomes][i][j]
    if sum(map(bool, row)) > 1:
        table.append(row)
pd.DataFrame(table, columns=['gender', 'city', 'income']).head(10)

,gender,city,income
0,,perth,28019.494
1,,melbourne,157033.920
2,female,,1231675.566
3,make,,115081.643
4,male,melbournd,67499.911
5,female,,81274.782
6,female,,62338.207
7,femalr,sydney,65742.741
8,male,melbovrne,157723.128
9,male,melbourne,


The last 20 groups look like this.

## Sneak peek at the result

We obviously can't do this in a real-world setting, but let's view the linkage using the PII. If the IDs match, then we are correct.

In [19]:
with open('data/dataset-alice.csv') as f:
    r = csv.reader(f)
    next(r)  # Skip header
    dataset_alice = tuple(r)
    
with open('data/dataset-bob.csv') as f:
    r = csv.reader(f)
    next(r)  # Skip header
    dataset_bob = tuple(r)
    
with open('data/dataset-charlie.csv') as f:
    r = csv.reader(f)
    next(r)  # Skip header
    dataset_charlie = tuple(r)

In [20]:
table = []
for group in linkage_groups:
    for i, j in sorted(group):
        table.append([dataset_alice, dataset_bob, dataset_charlie][i][j])
    table.append([''] * 6)
    
pd.DataFrame(table, columns=['id', 'given name', 'surname', 'dob', 'phone number', 'non-linking']).tail(15)



,id,given name,surname,dob,phone number,non-linking
6450,,,,,,
6451,6686,soph'le,thrushell,26-07-1980,02 2990 1023,femake
6452,6686,stophie,thrus5ell,26-07-1080,02 2990 1023,brisbane
6453,6686,sophi,thrussell,26-07-1980,02 2990 2023,33833.975
6454,,,,,,
6455,9875,danielJ,muenchow,31-02-1933,,male
6456,9875,danielle,muenchow,31-02-1933,03 7156 8980,melbourne
6457,,,,,,
6458,8433,julia,carbone,16-01-1969,07 9019 8256,male
6459,8433,juli,carbone,16-01-1969,07 9019 8356,sydnev


In [22]:
# Deleting the project
!anonlink delete-project --project="{credentials['project_id']}" \
        --apikey="{credentials['result_token']}" \
        --server="{SERVER}"

Project deleted
